<a href="https://colab.research.google.com/github/asimaghami/Classification/blob/main/2_AECOPD__Untitled1_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install tensorflow-probability
!pip install numpy

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam



In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
#data = pd.read_csv('https://raw.githubusercontent.com/harpreetSinghGuller/Data-Science-R/master/SAHeart.csv')
df = pd.read_csv("AECOPD.csv")
#df["famhist"] = df["famhist"].map({"Absent": 0, "Present": 1})


# Split the dataset into features and target
X = df.drop('Group', axis=1)
y = df['Group']

#Drop any missing values
df.dropna(inplace=True)


# Normalize the features
## X = (X - X.mean(axis=0)) / X.std(axis=0)



In [23]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [24]:
# Convert the inputs to the expected data type
X_train = tf.cast(X_train, tf.float32)
y_train = tf.cast(y_train, tf.float32)
X_test = tf.cast(X_test, tf.float32)
y_test = tf.cast(y_test, tf.float32)

In [25]:
np.random.seed(0)
tf.random.set_seed(0)


tfd = tfp.distributions
tfb = tfp.bijectors

In [26]:
num_layers = 20
my_bijects = []
ndim = 12  # Adjust this to match the number of features in your dataset

zdist = tfd.MultivariateNormalDiag(loc=[0.0] * ndim)

# loop over desired bijectors and put into list
for i in range(num_layers):
    # Syntax to make a MAF
    anet = tfb.AutoregressiveNetwork(
        params=ndim, hidden_units=[128, 128], activation="relu"
    )
    ab = tfb.MaskedAutoregressiveFlow(anet)
    # Add bijector to list
    my_bijects.append(ab)
    # Now permuate (!important!)
    permute = tfb.Permute(list(range(ndim-1, -1, -1)))  # Adjust this to match the number of features in your dataset
    my_bijects.append(permute)

# put all bijectors into one "chain bijector"
# that looks like one
big_bijector = tfb.Chain(my_bijects)

# make transformed dist
td = tfd.TransformedDistribution(zdist, bijector=big_bijector)

In [27]:
import tensorflow as tf

# Define the loss function
negloglik = lambda y, rv_y: -rv_y.log_prob(y)

# Define the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(19),
  tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., :1],
                                                     scale=1e-5 + tf.math.softplus(0.05 * t[...,1:]))),
])

# Compile the model
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), loss=negloglik)

# Train the model
model.fit(X_train, y_train, epochs=100, verbose=False)


In [28]:
# Evaluate the model
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 4ms/step - loss: nan


nan

In [29]:
# Make predictions
y_pred = model.predict(X_test)

4/4 [==============================] - 0s 5ms/step


In [30]:
y_pred_binary = np.where(y_pred > 0.5, 1, 0)


In [31]:
y_pred_binary = np.argmax(y_pred_binary, axis=1)

In [32]:
y_pred_binary.shape

(104,)

In [33]:
y_test

<tf.Tensor: shape=(104,), dtype=float32, numpy=
array([1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0.,
       1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0.,
       1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1.,
       0., 0.], dtype=float32)>

In [34]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_binary))


              precision    recall  f1-score   support

         0.0       0.60      1.00      0.75        62
         1.0       0.00      0.00      0.00        42

    accuracy                           0.60       104
   macro avg       0.30      0.50      0.37       104
weighted avg       0.36      0.60      0.45       104



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
# Gaussian Mixture Model (GMM)
gmm = GaussianMixture(n_components=5)
gmm.fit(X_train_scaled)

# Support Vector Machine (SVM)
svm = SVC()
svm.fit(X_train_scaled, y_train)

# Logistic Regression (LR)
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)

# Random Forest (RF)
rf = RandomForestClassifier()
rf.fit(X_train_scaled, y_train)

# MLP (Multi-Layer Perceptron)
mlp = MLPClassifier()
mlp.fit(X_train_scaled, y_train)

# XGBoost
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train_scaled, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [36]:
# Evaluate accuracy
models = {
    "GMM": gmm,
    "SVM": svm,
    "LR": lr,
    "RF": rf,
    "MLP": mlp,
    "XGBoost": xgb_model
}

accuracies = {}

for model_name, model in models.items():
    if model_name == "LDA":
        X_test_transformed = model.transform(X_test_scaled)
    else:
        X_test_transformed = X_test_scaled
    y_pred = model.predict(X_test_transformed)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies[model_name] = accuracy

# Print accuracies
for model_name, accuracy in accuracies.items():
    print(f"{model_name}: {accuracy}")

GMM: 0.25961538461538464
SVM: 0.9807692307692307
LR: 0.9807692307692307
RF: 1.0
MLP: 0.9903846153846154
XGBoost: 1.0
